In [2]:
import time
# Start tracking process time
start_time = time.process_time()

In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [4]:
# Read the drug interaction data
df_interaction = pd.read_csv('C://Users/Peng/Downloads/ddi.csv')
df_interaction = df_interaction.dropna()

In [5]:
# Remove duplicates
mask = df_interaction.duplicated(subset=['drugbank_id1', 'drugbank_id2']) | df_interaction.duplicated(subset=['drugbank_id2', 'drugbank_id1'])
df_interaction = df_interaction[~mask]

In [ ]:
# remove duplicates if drug1 = drug2 and drug2 = drug1
mask = df_interaction.duplicated(subset=['drugbank_id1', 'drugbank_id2']) | df_interaction.duplicated(subset=['drugbank_id2', 'drugbank_id1'])

# Remove duplicates
df_interaction = df_interaction[~mask]

df_interaction

In [6]:
# Define values to remove
values_to_remove = [
    '(4-{(2S)-2-[(tert-butoxycarbonyl)amino]-3-methoxy-3-oxopropyl}phenyl)methaneseleninic acid',
    '(S)-2-Amino-3-(4h-Selenolo[3,2-B]-Pyrrol-6-Yl)-Propionic Acid',
    'Florbetaben (18F)']

In [7]:
# Remove rows with values to remove
df_interaction = df_interaction[~(df_interaction['drug1'].str.strip().isin(values_to_remove) | df_interaction['drug2'].str.strip().isin(values_to_remove))]

# Read the drugList data
res = pd.read_csv('output_data.csv')
drugList_A = res.drugA.unique().tolist()
drugList_B = res.drugB.unique().tolist()
drugList_A.extend(drugList_B)
drugList = [*set(drugList_A)]
print(len(drugList))

3475


In [8]:
drugList_A = df_interaction.drug1.unique().tolist()
print(len(drugList_A))
drugList_B = df_interaction.drug2.unique().tolist()
print(len(drugList_B))
drugList_A.extend(drugList_B)
print(len(drugList_A))
drugList = [*set(drugList_A)]
print(len(drugList))

4389
4390
8779
4390


In [9]:
# Create a DataFrame to store the drugList results
# Modify this part to include empty drugbank IDs
drug_id_list = []
for drug_name in drugList:
    drug_id1 = df_interaction.loc[df_interaction['drug1'] == drug_name, 'drugbank_id1'].values
    drug_id2 = df_interaction.loc[df_interaction['drug2'] == drug_name, 'drugbank_id2'].values
    if drug_id1.any():
        drug_id_list.append(drug_id1[0])
    elif drug_id2.any():
        drug_id_list.append(drug_id2[0])
    else:
        drug_id_list.append('N/A')

drugList_df = pd.DataFrame({'index': range(len(drugList)),
                            'drug_id': drug_id_list,
                            'drug_name': drugList})

# Save the drug information DataFrame to a CSV file
drugList_df.to_csv('drugList062124.csv', index=False)

In [ ]:
# Create a DataFrame to store the drugList results
# Modify this part to include empty drugbank IDs
drugbank_id_list = []
for drug_name in drugList:
    drugbank_id1 = df_interaction.loc[df_interaction['drug1'] == drug_name, 'drugbank_id1'].values
    drugbank_id2 = df_interaction.loc[df_interaction['drug2'] == drug_name, 'drugbank_id2'].values
    if drugbank_id1.any():
        drugbank_id_list.append(drugbank_id1[0])
    elif drug_id2.any():
        drugbank_id_list.append(drug_id2[0])
    else:
        drugbank_id_list.append('N/A')

drugList_df = pd.DataFrame({'index': range(len(drugList)),
                            'drugbank_id': drugbank_id_list,
                            'drug_name': drugList})

# Save the drug information DataFrame to a CSV file
drugList_df.to_csv('drugList 061524.csv', index=False)

In [ ]:
# Read the SMILES data: for 11583 drugbank IDs
drugbank_smiles = pd.read_csv("C://Users/Peng/Downloads/SMILES Aug 2023rev1.0.csv", encoding='utf-8') # Don't alter file name

# Merge drugList data with SMILES data
drug_info_df = pd.merge(drugList_df, drugbank_smiles, left_on='drugbank_id', right_on='drugbank_id', how='left')

# Fill empty SMILES with a placeholder (e.g., 'N/A' or any desired value)
drug_info_df['smiles'].fillna('N/A', inplace=True)# Replace empty drugbank IDs with correct drugbank IDs

# Replace empty drugbank IDs with correct drugbank IDs
drug_info_df['drugbank_id'].fillna('N/A', inplace=

In [12]:
# Read the filtered database with the "target" column
df_filtered = pd.read_csv('C://Users/Peng/Downloads/DRKG Oct 2023/filtered_database 110423.csv')

# Merge the "target" column with drug information based on "drugbank_id" and "drug_name"
drug_info_df = pd.read_csv('C://Users/Peng/Downloads/DRKG Oct 2023/SMILES-3475 sorted by drug IDs.csv')
drug_info_df = pd.merge(drug_info_df, df_filtered[['drugbank_id', 'drug_name', 'target','enzyme']], on=['drugbank_id', 'drug_name'], how='left')

# Fill empty "target" values with a placeholder (e.g., 'N/A' or any desired value)
drug_info_df['target'].fillna('N/A', inplace=True)

# Fill empty SMILES with 'N/A'
drug_info_df['smiles'].fillna('N/A', inplace=True)

# Select and reorder columns
# drug_info_df = drug_info_df[['index', 'drugbank_id', 'target', 'enzyme', 'smiles', 'drug_name']]

drug_info_df = drug_info_df[['drugbank_id', 'target', 'enzyme', 'smiles', 'drug_name']]


# Save the updated drug information DataFrame to a CSV file
drug_info_df.to_csv('C://Users/Peng/Downloads/DRKG Oct 2023/drug_information.csv', index=False)

In [13]:
# End timer and print elapsed time
end_time = time.process_time()
print(f"Processing time: {end_time - start_time} seconds") # 41.5 minutes process time

Processing time: 1096.640625 seconds
